In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
out_dir = "./data/out"
in_dir = "./data/in"

In [5]:
optimizer_name = "brute force"
set_names = ["set-1", "set-2", "set-3"]
optimizer_dir = os.path.join(out_dir, optimizer_name)

In [16]:
for set_name in set_names:
    set_dir = os.path.join(optimizer_dir, set_name)
    experiment_dir = os.path.join(set_dir, "01")
    settings_path = os.path.join(experiment_dir, "settings.json")
    
    settings_file = open(settings_path, "r")
    settings_doc = json.loads(settings_file.read())
    
    resampling_period = settings_doc["resampling"]["period[min]"]
    
    search_space_doc = settings_doc["search space"][0]
    print(search_space_doc[1])
    levels_doc = search_space_doc[0]["levels"]
    levels_unique_count = levels_doc["unique count"]
    levels_lower_bound = levels_doc["lower bound"]
    
    print(settings_doc)
    assert(False)

{'count': 3, 'lower bound': '15/20', 'unique count': 15}


TypeError: string indices must be integers

In [7]:
set_dirs

['./data/out/brute force/set-1',
 './data/out/brute force/set-2',
 './data/out/brute force/set-3']